In [4]:
%run ./bk.ipynb

In [5]:
from binance import Client

In [6]:
# import required libraries
import pandas as pd
import ta
import numpy as np
import time

In [7]:
# access platform using API
client = Client(ApKey,SecKey)

In [8]:
client.futures_account()['positions']

[{'symbol': 'RAYUSDT',
  'initialMargin': '0',
  'maintMargin': '0',
  'unrealizedProfit': '0.00000000',
  'positionInitialMargin': '0',
  'openOrderInitialMargin': '0',
  'leverage': '25',
  'isolated': True,
  'entryPrice': '0.0',
  'maxNotional': '5000',
  'positionSide': 'BOTH',
  'positionAmt': '0.0',
  'notional': '0',
  'isolatedWallet': '0',
  'updateTime': 1635500578297,
  'bidNotional': '0',
  'askNotional': '0'},
 {'symbol': 'API3USDT',
  'initialMargin': '0',
  'maintMargin': '0',
  'unrealizedProfit': '0.00000000',
  'positionInitialMargin': '0',
  'openOrderInitialMargin': '0',
  'leverage': '25',
  'isolated': True,
  'entryPrice': '0.0',
  'maxNotional': '5000',
  'positionSide': 'BOTH',
  'positionAmt': '0.0',
  'notional': '0',
  'isolatedWallet': '0',
  'updateTime': 1645577738323,
  'bidNotional': '0',
  'askNotional': '0'},
 {'symbol': 'SUSHIUSDT',
  'initialMargin': '0',
  'maintMargin': '0',
  'unrealizedProfit': '0.00000000',
  'positionInitialMargin': '0',
  'o

In [9]:
def getfuturesdata(symbol, interval, lookback):
    futuresframe = pd.DataFrame(client.futures_historical_klines(symbol, interval,lookback + ' hour ago UTC'))
    futuresframe = futuresframe.iloc[:,:6]
    futuresframe.columns= ['Time','Open','High','Low','Close','Volume']
    futuresframe = futuresframe.set_index('Time')
    futuresframe.index= pd.to_datetime(futuresframe.index, unit='ms')
    futuresframe= futuresframe.astype(float)
    return futuresframe

In [10]:
df=getfuturesdata('ADAUSDT','1m','2')

In [11]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-08-18 16:03:00,0.5370,0.5370,0.5362,0.5363,388049.0
2022-08-18 16:04:00,0.5362,0.5367,0.5361,0.5366,360814.0
2022-08-18 16:05:00,0.5366,0.5370,0.5363,0.5364,301104.0
2022-08-18 16:06:00,0.5363,0.5364,0.5354,0.5359,414132.0
2022-08-18 16:07:00,0.5359,0.5359,0.5354,0.5356,119339.0
...,...,...,...,...,...
2022-08-18 17:58:00,0.5266,0.5270,0.5264,0.5265,486257.0
2022-08-18 17:59:00,0.5266,0.5272,0.5265,0.5268,341074.0
2022-08-18 18:00:00,0.5268,0.5278,0.5264,0.5278,1005900.0


In [12]:
def applytechnicals(df):
    df['%k']=ta.momentum.stoch(df.High,df.Low,df.Close,window=14,smooth_window=3)
    df['%D']=df['%k'].rolling(3).mean()
    df['rsi']= ta.momentum.rsi(df.Close, window=14)
    df['macd']= ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)
    return df

In [13]:
df=applytechnicals(df)

In [14]:
df

,Open,High,Low,Close,Volume,%k,%D,rsi,macd
Time,,,,,,,,,
2022-08-18 16:36:00,0.5283,0.5291,0.5281,0.5290,1252273.0,25.925926,25.696239,29.759260,-0.000418
2022-08-18 16:37:00,0.5290,0.5292,0.5285,0.5287,1047726.0,22.500000,21.568332,28.836905,-0.000366
2022-08-18 16:38:00,0.5288,0.5288,0.5264,0.5272,2738800.0,9.876543,19.434156,24.712615,-0.000396
2022-08-18 16:39:00,0.5272,0.5279,0.5255,0.5277,2550077.0,26.506024,19.627522,28.389183,-0.000347
2022-08-18 16:40:00,0.5278,0.5278,0.5268,0.5272,1148880.0,20.481928,18.954832,26.970784,-0.000315
...,...,...,...,...,...,...,...,...,...
2022-08-18 17:58:00,0.5266,0.5270,0.5264,0.5265,486257.0,5.000000,5.000000,27.313520,-0.000226
2022-08-18 17:59:00,0.5266,0.5272,0.5265,0.5268,341074.0,12.500000,8.333333,31.368235,-0.000177
2022-08-18 18:00:00,0.5268,0.5278,0.5264,0.5278,1005900.0,40.540541,19.346847,42.818722,-0.000065


In [15]:
class Signals:
    
    def __init__(self,df,lags):
        self.df=df
        self.lags=lags
        
    def getbuytrigger(self):
        dfx=pd.DataFrame()
        for i in range(self.lags + 1):
            mask=(self.df['%k'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx= dfx.append(mask, ignore_index=True)
        return dfx.sum(axis=0)
    
    def getselltrigger(self):
        dfy=pd.DataFrame()
        for i in range(self.lags + 1):
            mask=(self.df['%k'].shift(i) > 80) & (self.df['%D'].shift(i) > 80)
            dfy= dfy.append(mask, ignore_index=True)
        return dfy.sum(axis=0)
    
    def decide(self):
        self.df['buytrigger']=np.where(self.getbuytrigger(),1,0)
        self.df['Buy']=np.where((self.df.buytrigger) & (self.df['%k'].between(20,80)) 
                               & (self.df['%D'].between(20,80)) & (self.df.rsi > 50)
                               & (self.df.macd > 0),1,0)
        
        self.df['selltrigger']=np.where(self.getselltrigger(),1,0)
        self.df['Sell']=np.where((self.df.selltrigger) & (self.df['%k'].between(20,80)) 
                               & (self.df['%D'].between(20,80)) & (self.df.rsi < 50)
                               & (self.df.macd < 0),1,0)
        
        

In [16]:
inst =Signals(df,5)

In [17]:
inst.decide()

C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

In [18]:
df

,Open,High,Low,Close,Volume,%k,%D,rsi,macd,buytrigger,Buy,selltrigger,Sell
Time,,,,,,,,,,,,,
2022-08-18 16:36:00,0.5283,0.5291,0.5281,0.5290,1252273.0,25.925926,25.696239,29.759260,-0.000418,0,0,0,0
2022-08-18 16:37:00,0.5290,0.5292,0.5285,0.5287,1047726.0,22.500000,21.568332,28.836905,-0.000366,0,0,0,0
2022-08-18 16:38:00,0.5288,0.5288,0.5264,0.5272,2738800.0,9.876543,19.434156,24.712615,-0.000396,1,0,0,0
2022-08-18 16:39:00,0.5272,0.5279,0.5255,0.5277,2550077.0,26.506024,19.627522,28.389183,-0.000347,1,0,0,0
2022-08-18 16:40:00,0.5278,0.5278,0.5268,0.5272,1148880.0,20.481928,18.954832,26.970784,-0.000315,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-18 17:58:00,0.5266,0.5270,0.5264,0.5265,486257.0,5.000000,5.000000,27.313520,-0.000226,1,0,0,0
2022-08-18 17:59:00,0.5266,0.5272,0.5265,0.5268,341074.0,12.500000,8.333333,31.368235,-0.000177,1,0,0,0
2022-08-18 18:00:00,0.5268,0.5278,0.5264,0.5278,1005900.0,40.540541,19.346847,42.818722,-0.000065,1,0,0,0


In [19]:
def strategy(pair, qty):
    df= getfuturesdata(pair,'1m','5')
    applytechnicals(df)
    inst= Signals(df,25)
    inst.decide()
    print(f'current Close is '+str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1]:
        client.futures_change_leverage(symbol=pair,leverage=2)
        order = client.futures_create_order(symbol=pair,
                                   side='BUY',
                                   type='MARKET',
                                   quantity=qty)
        print(order)
        currentbuyprice=df.Close.iloc[-1]
        print(currentbuyprice)
        time.sleep(0.5)
        takeprofit='currentbuyprice*1.005'
        print(f'current Target is '+ takeprofit)
        if df.CLose[-1] >= buyprice* 1.005:
            order = client.futures_create_order(symbol=pair,
                                   side='SELL',
                                   type='TAKE_PROFIT',
                                   stopPrice=takeprofit,closePosition='True')
            print(order)
        

### def getpositions():
    
    try:
        a = client.futures_account()['positions']
        for i in a:
            
            amount = float((i['positionAmt']))
            
            if amount != 0:
                pair=i['symbol']
                amt=amount
                
                print(amt,pair)
            
    except Exception as e:
        print(e)
    return pair,amt

In [20]:
strategy('WAVESUSDT',1)

C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

current Close is 5.612


C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfy= dfy.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfy= dfy.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfy= dfy.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_5064\1747439584.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfy= dfy.append(mask, ignore_index=True)


In [107]:
while True:
    strategy('WAVESUSDT',1)
    time.sleep(0.1)

C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

current Close is 5.763


C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

current Close is 5.763


C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

...

current Close is 5.766


C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

current Close is 5.765


C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local\Temp\ipykernel_6112\1747439584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx= dfx.append(mask, ignore_index=True)
C:\Users\okori\AppData\Local

KeyboardInterrupt: 